# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
full_data_rows_list = [] 
    

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
   
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

complete_file = 'complete_event_datafile.csv'

with open(complete_file, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length', 'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# Confirm number of rows in csv file
with open(complete_file, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
from cassandra.cluster import Cluster
cluster = Cluster()
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [24]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_db
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
try:
    session.set_keyspace('music_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

This query returns the artist, song and song lenth based on a songplay (itemInSession) during a session. It uses sessionId and itemInSession as the primary key to ensure each unique song play within a session is accessible.

In [26]:
# Query for testing
songs_test_query = "SELECT artist, song, length FROM songs WHERE sessionId = 338 AND itemInSession = 4"

# Table creation
create_songs = "CREATE TABLE IF NOT EXISTS songs "
create_songs = create_songs + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_songs)
except Exception as e:
    print(e)          

In [27]:
# Load data into table
with open(complete_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        songs_query = "INSERT INTO songs (sessionId, itemInSession, artist, song, length) "
        songs_query = songs_query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(songs_query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [28]:
# View correct data with testing query
try:
    rows = session.execute(songs_test_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


---
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

This query provides the artist and song as well as the user who listened to it based on a userId and sessionId. The query uses userId and sessionId as partition key for rapid searching and itemInSession as the clustering column to sort the data correctly. 

In [29]:
# Query for testing
user_songs_test_query = "SELECT artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182"

# Table creation
create_user_songs = "CREATE TABLE IF NOT EXISTS user_songs "
create_user_songs = create_user_songs + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(create_user_songs)
except Exception as e:
    print(e)                   

In [30]:
# Load data into table
with open(complete_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user_songs_query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        user_songs_query = user_songs_query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(user_songs_query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [31]:
# View correct data with testing query
try:
    rows = session.execute(user_songs_test_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


---
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

This query provides every unique user first and last name who listened to a particular song.
It uses song as the partition key with userId as the clustering column so that every unique user and song combination will be provided in the returned results.

In [32]:
# Query for testing
song_plays_test_query = "SELECT firstName, lastName FROM song_plays WHERE song = 'All Hands Against His Own'"

# Table creation
create_song_plays = "CREATE TABLE IF NOT EXISTS song_plays "
create_song_plays = create_song_plays + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(create_song_plays)
except Exception as e:
    print(e)                  

In [33]:
# Load data into table
with open(complete_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        song_plays_query = "INSERT INTO song_plays (song, userId,  firstName, lastName) "
        song_plays_query = song_plays_query + "VALUES (%s, %s, %s, %s)"
        session.execute(song_plays_query, (line[9], int(line[10]), line[1], line[4]))

In [34]:
# View correct data with testing query
try:
    rows = session.execute(song_plays_test_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [35]:
session.execute("DROP TABLE IF EXISTS songs")
session.execute("DROP TABLE IF EXISTS user_songs")
session.execute("DROP TABLE IF EXISTS song_plays")

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()